In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\CNS.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
Init();


In [2]:
BoSSSshell.WorkflowMgm.Init("CNS_AcousticWave1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 12; Grid Count = 12; Path = \\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D }

In [3]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\CNS_AcousticWave1D

In [4]:
var db=OpenOrCreateDatabase(dbPath);
var ses= db.Sessions.Where(si => si.Name.Contains("xCells61") && si.Name.Contains("1sinus"));
ses

#0: AcousticWave	AW_p3_xCells61_yCells3_sP0.5_pST10_wP3.2_ampneg1E-05_amppos0_wL0.8_Mach1.5_1sinus	4/16/2024 7:26:38 AM	50d438b1...
#1: AcousticWave	AW_p3_xCells61_yCells3_sP1.5_pST10_wP-0.4_ampneg1E-05_amppos0_wL0.8_Mach1.5_1sinus	4/16/2024 7:27:55 AM	2b325244...
#2: AcousticWave	AW_p3_xCells61_yCells3_sP1.5_pST10_wP-0.4_ampneg0_amppos1E-05_wL0.8_Mach1.5_1sinus	4/16/2024 7:25:22 AM	39a710e5...


In [6]:
(double[] xVals, double[]yVals, double tPhys) GetPlot(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si, double t0){
//double shockPosition=0.5;
var p1= new double[] {xMin,0.0015};
var p2=new double[] {xMax,0.0015};

//closest timestep to time t
var tiEnd=si.Timesteps.Where(ti =>ti.PhysicalTime >t-1e-2).OrderBy(ti => Math.Abs(ti.PhysicalTime - t))
            .FirstOrDefault();

var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);

// substract the t0 Base to only obtain the pertuabtions
var ti=si.Timesteps.Where(ti =>Math.Abs(ti.PhysicalTime - t0)<1e-1).OrderBy(ti => Math.Abs(ti.PhysicalTime - t0))
            .FirstOrDefault();
var tOField=ti.GetField(field);
double[] yVals_tOField=tOField.EvaluateAlongLine(p1,p2,nRef);
yVals.AccV(-1.0,yVals_tOField);

double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals,tiEnd.PhysicalTime);
}

In [7]:
double eps=0.000001;
double xMin=0.0+eps;double xMax=3.0-eps; int xRef=100;
//double tMin =15.87; double tMax=18.87;double t0=9.99;int tRef=6; downstream Mach1.5 wL0.4, wp3.2
//double tMin =11.2; double tMax=tMin+3.0;double t0=9.95;int tRef=3; upstream Mach1.5 wL0.4, wp0.3
double tMin =10.0; double tMax=tMin+8.0;double t0=9.95;int tRef=6;
int nRows=2;
bool doPlt=false;

## Pressure perturbations

In [12]:
Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[(tRef+1)/nRows,nRows];
foreach(var si in ses){
    string field="p";
    string csvFileName = si.Name + "_p_per.csv";
    // Format
    var fmt= new PlotFormat();
    fmt.WithPointSize(0.0);
    fmt.LineColor=LineColors.Red;

    
    using (StreamWriter writer = new StreamWriter(csvFileName)){
        for(int Ti=0;Ti< tRef;Ti++){
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="p'";
            var t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
            //Console.WriteLine(t);
            double[] xVals=new double[xRef],yVals=new double[xRef];
            double tPhys=t;
            try{ 
                ( xVals,yVals,tPhys)=GetPlot(xMin,xMax,t,xRef,field, si,t0);
            }catch{
                Console.WriteLine($"Failed for t={t}");
            }
            plot.AddDataGroup("t="+Math.Round(tPhys,2),xVals,yVals,fmt);
            multiplotsPressure[Ti/nRows,Ti %nRows]=plot;
            if (Ti == 0)
                {
                    // Write header row
                    writer.WriteLine($"x/t,{string.Join(",", xVals)}");
                }
                // Write data row
                writer.WriteLine($"{Math.Round(tPhys, 2)},{string.Join(",", yVals)}");     
            }
    };
 }        


In [97]:
 Array.ForEach(multiplotsPressure.Cast<Plot2Ddata>().ToArray(), 
     plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax =  1.5*1e-5; });
//multiplotsPressure[1,0].YrangeMax = 5 * 1e-7; multiplotsPressure[1,0].YrangeMin = -6 * 1e-7;
 //multiplotsPressure[2,0].YrangeMax = 5 * 1e-7; multiplotsPressure[2,0].YrangeMin = -6 * 1e-7;
 //multiplotsPressure[2,1].YrangeMax = 5 * 1e-7; multiplotsPressure[2,1].YrangeMin = -6 * 1e-7;
multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5x10 -5 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 1.5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 p' 
 
 
 
 
 x 
 
 
 
 
 t=10.2 
 
 
 t=10.2 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M256.2,59.1 L309.6,59.1 M109.5,89.2 L110.9,89.3 L112.4,89.3 L113.8,89.2 L115.3,89.2 L116.7,89.3
 L118.1,89.3 L119.6,89.2 L121.0,89.0 L122.4,89.5 L123.9,90.3 L125.3,88.6 L126.8,80.7 L128.2,75.5
 L129.6,86.1 L131.1,108.2 L132.5,124.2 L133.9,121.9 L135.4,108.0 L136.8,88.0 L138.3,66.7 L139.7,58.1
 L141.1,66.3 L142.6,83.1 L144.0,92.7 L145.4,91.5 L146.9,87.5 L148.3,88.5 L149.8,89.9 L151.2,89.8
 L152.6,88.8 L154.1,89.2 L155.5,89.5 L156.9,89.2 L158.4,89.2 L159.8,89.3 L161.3,89.2 L162.7,89.2
 L164.1,89.3 L165.6,89.3 L167.0,89.2 L168.4,89.3 L169.9,89.3 L171.3,89.2 L172.8,89.3 L174.2,89.3
 L175.6,89.2 L177.1,89.3 L178.5,89.3 L179.9,89.2 L181.4,89.3 L182.8,89.3 L184.3,89.2 L185.7,89.3
 L187.1,89.3 L188.6,89.2 L190.0,89.3 L191.4,89.2 L192.9,89.2 L194.3,89.3 L195.8,89.2 L197.2,89.3
 L198.6,89.3 L200.1,89.2 L201.5,89.3 L202.9,89.3 L204.4,89.2 L205.8,89.3 L207.3,89.3 L208.7,89.3
 L210.1,89.3 L211.6,89.2 L213.0,89.3 L214.5,89.3 L215.9,89.2 L217.3,89.2 L218.8,89.2 L220.2,89.3
 L221.6,89.3 L223.1,89.2 L224.5,89.2 L226.0,89.3 L227.4,89.3 L228.8,89.2 L230.3,89.3 L231.7,89.3
 L233.1,89.2 L234.6,89.3 L236.0,89.2 L237.5,89.3 L238.9,89.3 L240.3,89.2 L241.8,89.3 L243.2,89.3
 L244.6,89.2 L246.1,89.3 L247.5,89.3 L249.0,89.3 L250.4,89.3 L251.8,89.3 L253.3,89.3 L254.7,89.3
 L256.1,89.3 L257.6,89.2 L259.0,89.2 L260.5,89.2 L261.9,89.3 L263.3,89.3 L264.8,89.3 L266.2,89.3
 L267.6,89.3 L269.1,89.3 L270.5,89.2 L272.0,89.2 L273.4,89.2 L274.8,89.3 L276.3,89.2 L277.7,89.2
 L279.1,89.2 L280.6,89.3 L282.0,89.3 L283.5,89.3 L284.9,89.3 L286.3,89.3 L287.8,89.3 L289.2,89.3
 L290.6,89.2 L292.1,89.2 L293.5,89.3 L295.0,89.3 L296.4,89.3 L297.8,89.3 L299.3,89.3 L300.7,89.3
 L302.2,89.2 L303.6,89.2 L305.0,89.2 L306.5,89.2 L307.9,89.3 L309.3,89.3 L310.8,89.3 L312.2,89.3
 L313.7,89.3 L315.1,89.2 L316.5,89.2 L318.0,89.2 L319.4,89.2 L320.8,89.2 L322.3,89.2 L323.7,89.3
 L325.2,89.2 L326.6,89.2 L328.0,89.2 L329.5,89.2 L330.9,89.2 L332.3,89.2 L333.8,89.2 L335.2,89.2
 L336.7,89.3 L338.1,89.3 L339.5,89.3 L341.0,89.3 L342.4,89.3 L343.8,89.3 L345.3,89.3 L346.7,89.3
 L348.2,89.3 L349.6,89.3 L351.0,89.2 L352.5,89.2 L353.9,89.2 L355.3,89.3 L356.8,89.3 L358.2,89.3
 L359.7,89.3 L361.1,89.2 L362.5,89.2 L364.0,89.2 L365.4,89.2 L366.8,89.2 L368.3,89.2 L369.7,89.3
 L371.2,89.3 L372.6,89.3 L374.0,89.2 L375.5,89.2 L376.9,89.3 L378.3,89.3 L379.8,89.3 L381.2,89.2
 L382.7,89.2 L384.1,89.3 L385.5,89.2 L387.0,89.2 L388.4,89.2 L389.8,89.2 L391.3,89.3 L392.7,89.3
 L394.2,89.3 L395.6,89.3 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.5x10 -5 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 1.5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 p' 
 
 
 
 
 x 
 
 
 
 
 t=10.49 
 
 
 t=10.49 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M695.1,59.1 L748.5,59.1 M559.5,89.2 L560.9,89.2 L562.4,89.2 L563.8,89.2 L565.3,89.2 L566.7,89.2
 L568.1,89.2 L569.6,89.2 L571.0,89.2 L572.4,89.2 L573.9,89.2 L575.3,89.2 L576.8,89.2 L578.2,89.3
 L579.6,89.3 L581.1,89.3 L582.5,89.3 L583.9,89.3 L585.4,89.2 L586.8,89.2 L588.3,89.3 L589.7,89.2
 L591.1,89.2 L592.6,89.3 L594.0,89.3 L595.4,89

## Density perturbations

In [98]:
foreach(var si in ses){
string field="rho";
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

Plot2Ddata[,] multiplotsDensity = new Plot2Ddata[(tRef+1)/nRows,nRows];
string csvFileName = si.Name + "rho_per.csv";
using (StreamWriter writer = new StreamWriter(csvFileName)){
    for(int Ti=0;Ti< tRef;Ti++){
        var plot= new Plot2Ddata();
        plot.Xlabel="x";
        plot.Ylabel="rho'";
        var t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
        //Console.WriteLine(t);
        double[] xVals=new double[xRef],yVals=new double[xRef];
        double tPhys=t;
        try{ 
            ( xVals,yVals,tPhys)=GetPlot(xMin,xMax,t,xRef,field, si,t0);
        }catch{
            Console.WriteLine($"Failed for t={t}");
        }
        plot.AddDataGroup("t="+Math.Round(tPhys,2),xVals,yVals,fmt);
        multiplotsDensity[Ti/nRows,Ti %nRows]=plot;
        if (Ti == 0)
        {
            // Write header row
            writer.WriteLine($"x/t,{string.Join(",", xVals)}");
        }
        // Write data row
        writer.WriteLine($"{Math.Round(tPhys, 2)},{string.Join(",", yVals)}");
                
    }
};
}


In [99]:
 Array.ForEach(multiplotsDensity .Cast<Plot2Ddata>().ToArray(), 
     plot => { plot.YrangeMin = -1 * 1e-5; plot.YrangeMax = 1 * 1e-5; });
//     multiplotsDensity[1,1].YrangeMax = 5 * 1e-7; multiplotsDensity[1,1].YrangeMin = -6 * 1e-7;
// multiplotsDensity[2,0].YrangeMax = 5 * 1e-7; multiplotsDensity[2,0].YrangeMin = -6 * 1e-7;
// multiplotsDensity[2,1].YrangeMax = 5 * 1e-7; multiplotsDensity[2,1].YrangeMin = -6 * 1e-7;
 multiplotsDensity .ToGnuplot().PlotSVG(xRes:1000,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 rho' 
 
 
 
 
 x 
 
 
 
 
 t=10.2 
 
 
 t=10.2 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M305.7,59.1 L359.1,59.1 M93.4,89.2 L95.2,89.3 L96.9,89.3 L98.7,89.2 L100.5,89.2 L102.2,89.3
 L104.0,89.3 L105.8,89.2 L107.5,89.0 L109.3,89.5 L111.1,90.3 L112.8,88.5 L114.6,80.1 L116.4,74.5
 L118.1,85.8 L119.9,109.5 L121.7,126.7 L123.4,124.2 L125.2,109.4 L127.0,87.9 L128.7,65.1 L130.5,55.8
 L132.3,64.6 L134.0,82.7 L135.8,93.0 L137.6,91.7 L139.4,87.4 L141.1,88.4 L142.9,90.0 L144.7,89.8
 L146.4,88.8 L148.2,89.2 L150.0,89.5 L151.7,89.2 L153.5,89.2 L155.3,89.3 L157.0,89.2 L158.8,89.2
 L160.6,89.3 L162.3,89.3 L164.1,89.2 L165.9,89.3 L167.6,89.3 L169.4,89.2 L171.2,89.3 L172.9,89.3
 L174.7,89.2 L176.5,89.3 L178.2,89.3 L180.0,89.2 L181.8,89.3 L183.5,89.3 L185.3,89.2 L187.1,89.3
 L188.8,89.3 L190.6,89.2 L192.4,89.3 L194.1,89.2 L195.9,89.2 L197.7,89.3 L199.4,89.2 L201.2,89.3
 L203.0,89.3 L204.7,89.2 L206.5,89.3 L208.3,89.3 L210.0,89.3 L211.8,89.3 L213.6,89.2 L215.3,89.3
 L217.1,89.3 L218.9,89.2 L220.6,89.2 L222.4,89.3 L224.2,89.3 L226.0,89.2 L227.7,89.2 L229.5,89.3
 L231.3,89.3 L233.0,89.3 L234.8,89.3 L236.6,89.3 L238.3,89.3 L240.1,89.3 L241.9,89.3 L243.6,89.3
 L245.4,89.3 L247.2,89.3 L248.9,89.2 L250.7,89.3 L252.5,89.2 L254.2,89.2 L256.0,89.2 L257.8,89.3
 L259.5,89.2 L261.3,89.3 L263.1,89.3 L264.8,89.3 L266.6,89.3 L268.4,89.3 L270.1,89.3 L271.9,89.3
 L273.7,89.3 L275.4,89.2 L277.2,89.2 L279.0,89.2 L280.7,89.3 L282.5,89.3 L284.3,89.3 L286.0,89.3
 L287.8,89.2 L289.6,89.2 L291.3,89.2 L293.1,89.2 L294.9,89.3 L296.6,89.3 L298.4,89.2 L300.2,89.2
 L301.9,89.2 L303.7,89.2 L305.5,89.2 L307.2,89.2 L309.0,89.3 L310.8,89.3 L312.5,89.3 L314.3,89.3
 L316.1,89.2 L317.9,89.2 L319.6,89.3 L321.4,89.3 L323.2,89.3 L324.9,89.3 L326.7,89.3 L328.5,89.3
 L330.2,89.3 L332.0,89.3 L333.8,89.3 L335.5,89.3 L337.3,89.3 L339.1,89.3 L340.8,89.3 L342.6,89.2
 L344.4,89.2 L346.1,89.2 L347.9,89.2 L349.7,89.2 L351.4,89.2 L353.2,89.2 L355.0,89.2 L356.7,89.2
 L358.5,89.2 L360.3,89.2 L362.0,89.2 L363.8,89.2 L365.6,89.2 L367.3,89.2 L369.1,89.2 L370.9,89.2
 L372.6,89.3 L374.4,89.3 L376.2,89.3 L377.9,89.3 L379.7,89.3 L381.5,89.3 L383.2,89.3 L385.0,89.3
 L386.8,89.3 L388.5,89.3 L390.3,89.2 L392.1,89.2 L393.8,89.2 L395.6,89.2 L397.4,89.2 L399.1,89.2
 L400.9,89.2 L402.7,89.2 L404.5,89.2 L406.2,89.2 L408.0,89.2 L409.8,89.2 L411.5,89.2 L413.3,89.2
 L415.1,89.3 L416.8,89.3 L418.6,89.3 L420.4,89.3 L422.1,89.3 L423.9,89.3 L425.7,89.3 L427.4,89.3
 L429.2,89.3 L431.0,89.3 L432.7,89.3 L434.5,89.3 L436.3,89.3 L438.0,89.3 L439.8,89.2 L441.6,89.2
 L443.3,89.2 L445.1,89.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1x10 -5 
 
 
 
 
 -5x10 -6 
 
 
 
 
 0 
 
 
 
 
 5x10 -6 
 
 
 
 
 1x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 rho' 
 
 
 
 
 x 
 
 
 
 
 t=10.49 
 
 
 t=10.49 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M794.6,59.1 L848.0,59.1 M593.4,89.2 L595.2,89.2 L596.9,89.2 L598.7,89.2 L600.5,89.2 L602.2,89.2
 L604.0,89.2 L605.8,89.2 L607.5,89.2 L609.3,89.2 L611.1,89.2 L612.8,89.2 L614.6,89.2 L616.4,89.3
 L618.1,89.3 L619.9,89.3 L621.7,89.3 L623.4,89.3 L625.2,89.2 L627.0,89.2 L628.7,89.3 L630.5,89.2
 L632.3,89.2 L634.0,89.2 L635.8,89.2 L637.6,89.2 L639.4,89.2 L641.1,89.2 L642.9,89.2 L644.7,89.2
 L646.4,89.3 L648.2,89.3 L650.0,89.3 L651.7,89.3 L653.5,89.3 L655.3,89.

## Determine the Amplification Factor for different Mach numbers 

In [100]:
var sM= db.Sessions.Where(si => si.Name.Contains("0.0001"));
sM

In [101]:
var MachNumbers = new int[] {2,3,4,5,6,7,8};
var i=4;
var Mach=MachNumbers[i];
var si=sM.Where(si => si.Name.Contains("Mach" + Mach.ToString())).SingleOrDefault();
si.ID

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#102.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [102]:
si.Timesteps.Last()

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#103.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)